In [2]:
import pandas as pd
import numpy as np
import re
import os
import requests
import time
from PIL import Image, ImageOps

https://flatuicolors.com/palette/ca

In [3]:
pd.set_option('display.max_columns', None)

### <span style="color: #95a5a6">Get Club Logos</span>

In [147]:
# read file containing logo file names from https://github.com/eddwebster/football_analytics/tree/master/img/club_badges/england
logo_files = pd.read_csv('data/team_to_logo_path.csv')
logo_files.dropna(subset=['Logo Path'], inplace=True)

In [150]:
# function to download and save image
def download_image(url, image_name):
    response = requests.get(url)
    with open(f'images/{image_name}', 'wb') as f:
        f.write(response.content)

#### <span style="color: #95a5a6">Get logos from Edd Webster Repo</span>

In [163]:
git_url = 'https://raw.githubusercontent.com/eddwebster/football_analytics/refs/heads/master/img/club_badges/england/{}'
# iterate through the list of URLs
for i, row in logo_files.iterrows():
    # get team name
    team_name = row['Team Name']
    # get the image filename
    image_name = row['Logo Path']
    image_url = git_url.format(image_name)
    # get image file type
    image_ext = image_name.split('.')[-1]
    save_as_name = team_name + '.' + image_ext
    # check if image exists
    if save_as_name in os.listdir('images'):
        continue
    # download and save the image
    download_image(image_url, save_as_name)
    time.sleep(1)

In [167]:
no_images = [tn for tn in match_matrix['team_1'].unique().tolist() if tn not in [i.split('.')[0] for i in os.listdir('images')]]
print('Teams with no images')
print(no_images)

Teams with no images
[]


#### <span style="color: #95a5a6">Add background colour to images</span>

In [239]:
size_limit_kb = 128
quality = 95

In [243]:
output_path

'images/backgrounded/Burnley.png'

In [244]:
os.path.getsize(output_path)

670028

In [245]:
size_limit_kb * 1024

131072

In [18]:
for team_name in match_matrix['team_1'].unique():
    # check if done
    # if team_name in [i.split('.')[0] for i in os.listdir('images/backgrounded')]:
    #     print('skipping: ', team_name)
    #     continue
    # read image
    image = Image.open(f"images/{team_name}.png").convert("RGBA")
    background = Image.new("RGBA", image.size, (87, 101, 116, 255))
    composited = Image.alpha_composite(background, image)
    # create output path
    output_path = f"images/backgrounded/{team_name}.png"
    # ieratively reduce quality to meet tableau size limit
    composited.thumbnail((350, 350))
    composited.save(output_path, optimize=True)

In [19]:
no_images = [tn for tn in match_matrix['team_1'].unique().tolist() if tn not in [i.split('.')[0] for i in os.listdir('images/backgrounded')]]
print('Teams with no images')
print(no_images)

Teams with no images
[]


### <span style="color: #95a5a6">Read Data</span>

In [8]:
keep_columns = {
    'Division' : 'division',
    'MatchDate' : 'match_date',
    'MatchTime' : 'match_time',
    'HomeTeam' : 'home_team',
    'AwayTeam' : 'away_team',
    'FTHome' : 'ft_home',
    'FTAway' : 'ft_away',
    # 'FTResult' : 'ft_result',
    # 'HTHome' : 'ht_home',
    # 'HTAway' : 'ht_away',
    # 'HTResult':'ht_result'
}

In [9]:
matches_raw = pd.read_csv('data/Matches.csv')
all_columns = matches_raw.columns
matches_raw = matches_raw[keep_columns.keys()].rename(columns=keep_columns).copy()
matches_raw.shape

C:\Users\Hieye\AppData\Local\Temp\ipykernel_10152\205114520.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  matches_raw = pd.read_csv('data/Matches.csv')


(226755, 7)

### <span style="color: #95a5a6">Prep Stadium Data</span>

In [10]:
# code used to output a template file for manually getting stadium names and locations 
# team_locs_template = matches_unique['team_1'].drop_duplicates().to_frame()
# team_locs_template['ground_name'] = None
# team_locs_template['lat_long'] = None
# team_locs_template[~team_locs_template['home_team'].isin(team_locations['team_1'].tolist())].to_csv('output/missing_team_locations_generated_file.csv', index=False)
# team_locs_template.to_csv('output/team_locations_generated_file.csv', index=False)

In [11]:
team_locations = pd.read_csv('data/team_locations.csv')
team_locations[['lat', 'long']] = team_locations['lat_long'].str.split(', ').values.tolist()
team_locations.rename(columns={'home_team':'ground_team_name'}, inplace=True)

In [12]:
# team_locations[team_locations['lat_long'].duplicated(keep=False)]

### <span style="color: #95a5a6">Prep Match Data</span>

In [13]:
matches_unique = matches_raw[matches_raw['division'].isin(['E0', 'E1', 'E2', 'E3'])].reset_index(drop=True)
matches_unique['match_id'] = matches_unique.index

In [14]:
# create a data set where each match is represented twice: team_1 -> team_2 and team_2 -> team_1
matches_unique.rename(columns={'home_team':'team_1', 'away_team':'team_2', 'ft_home':'goals_team_1', 'ft_away':'goals_team_2'}, inplace=True)
matches_unique['duplicate_row'] = False
matches_flipped = matches_unique.rename(columns={'team_1' : 'team_2', 'team_2':'team_1', 'goals_team_1':'goals_team_2', 'goals_team_2':'goals_team_1'})
matches_flipped['duplicate_row'] = True
# combine the datasets
match_matrix = pd.concat([matches_unique, matches_flipped]).reset_index(drop=True)

In [15]:
# create measures - goal diff and w/l
match_matrix['goal_diff'] = match_matrix['goals_team_1'] - match_matrix['goals_team_2']
# get results
draws = match_matrix['goals_team_1'] == match_matrix['goals_team_2']
wins = match_matrix['goals_team_1'] > match_matrix['goals_team_2']
losses = match_matrix['goals_team_1'] < match_matrix['goals_team_2']
# label matches
match_matrix.loc[draws, 'result'] = 'D'
match_matrix.loc[wins, 'result'] = 'W'
match_matrix.loc[losses, 'result'] = 'L'
# give scores based on result
match_matrix['result_coded'] = match_matrix['result'].replace('D', 0).replace('L', -1).replace('W', 1)

In [16]:
# format dates
match_matrix['match_date'] = pd.to_datetime(match_matrix['match_date'], format='%Y-%m-%d')
# add year and month
match_matrix['year'] = match_matrix['match_date'].dt.year
match_matrix['month'] = match_matrix['match_date'].dt.month

In [17]:
# label seasons
for y, year_data in match_matrix.groupby(['year']):
    # split data by summer break
    year_start_season = year_data[year_data['month'] <= 5]
    year_end_season = year_data[year_data['month'] >= 8]
    # label data
    match_matrix.loc[year_start_season.index, 'season'] = '-'.join([str(y-1), str(y)[-2:]])
    match_matrix.loc[year_end_season.index, 'season'] = '-'.join([str(y), str(y + 1)[-2:]])

### <span style="color: #95a5a6">Join Location Data</span>

In [260]:
match_locations = match_matrix.merge(team_locations, left_on=['team_2'], right_on=['ground_team_name'], how='outer', validate='m:1')

### <span style="color: #95a5a6">Join HEX Map Data</span>

In [261]:
hex_areas = pd.read_csv('data/team_to_hex_area_mapping.csv')
# hex_areas.dropna(subset=['Team Name'], inplace=True)

In [262]:
duplicates = hex_areas[(hex_areas['Team Name'].notna()) & (hex_areas['Team Name'].duplicated())]
if not duplicates.empty:
    print('Duplicated teams')
    display(duplicates)

In [263]:
missing_names = [x for x in match_locations['team_1'].unique() if x not in hex_areas['Team Name'].tolist()]
if missing_names:
    print(f'Missing {len(missing_names)} names')

In [264]:
match_locations_hex = match_locations.merge(hex_areas, left_on=['team_2'], right_on=['Team Name'], how='outer')

### <span style="color: #95a5a6">Output</span>

In [265]:
match_locations_hex.to_csv('output/match_data.csv', index=False)